In [409]:
# scrape ingredients from www.vegrecipesofindia.com
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time

In [303]:
def get_links(url):
    
    
    page = urlopen(url)
    soup = BeautifulSoup(page)
    links = []
 
    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))
 
    return links


In [377]:
def get_ingredients(recipe, recipe_name):
    
    ingredient_tags = recipe.select(".wprm-recipe-ingredient")
    
    names = []
    amounts = []
    units = []
    notes = []
    
    for ingredient in ingredient_tags:
        
        try:
            name = ingredient.select(".wprm-recipe-ingredient-name")[0].get_text()
        except:
            print("No ingredient name found!")
        
        try:
            amount = [ingredient.select(".wprm-recipe-ingredient-amount")[0].get_text()
        except:
            amount = ""
        
        try:
            unit = ingredient.select(".wprm-recipe-ingredient-unit")[0].get_text()
        except:
            unit = ""
        
        try:
            note = ingredient.select(".wprm-recipe-ingredient-notes")[0].get_text()
        except:
            note = ""
    
        names.append(name)
        amounts.append(amount)
        units.append(unit)
        notes.append(note)
        
    # compile into a little dataframe
    df = pd.DataFrame({
            
            'name': names,
            'amount': amounts,
            'units': units,
            'notes': notes
    })
    
    df['recipe'] = recipe_name
    
    return df

In [404]:
def get_recipe_details(soup):
    
    #url
    url = soup.find("meta",  property="og:url")["content"]


    # name
    title = soup.find("meta",  property="og:title")["content"]

    name = title.partition('recipe')[0]

    # description
    description = title.partition('recipe')[2]

    # tags
    tag_tags = soup.find_all("meta", property="article:tag")
    tags = []
    for tag in tag_tags:
        tags.append(tag['content'])

    # sections
    section_tags = soup.find_all("meta", property="article:section")
    sections = []
    for section in section_tags:
        sections.append(section['content'])

    #categories
    categories = soup.find(class_='entry-categories').get_text()[13:].split(",")

    all_labels = tags + sections + categories
    labels = ",".join(list(set(all_labels)))
    

    # has photos
    has_photos = False
    if "step by step photos" in soup.find("meta", property="og:description")["content"]:
        has_photos = True

    ings = soup.find(class_ ="wprm-recipe-container")


    # prep time
    prep_time = [ings.get_text().partition('"prepTime":"')[2].partition('",')[0][2:-1]]

    # cook time
    cook_time = [ings.get_text().partition('"cookTime":"')[2].partition('",')[0][2:-1]]

    # total time
    total_time = [ings.get_text().partition('"totalTime":"')[2].partition('",')[0][2:-1]]

    # calories per serving
    calories = [ings.get_text().partition('"calories":"')[2].partition(' kcal')[0]]


    # course
    try:
        course = re.sub('\s+', '', soup.find(class_ ="wprm-recipe-course-container").get_text()).partition(":")[2]
    except:
        course = ''
        
    # cuisine
    try:
        cuisine = re.sub('\s+', '', soup.find(class_ ="wprm-recipe-cuisine-container").get_text()).partition(":")[2]
    except:
        cuisine = ''
        
    # no of servings
    try:
        servings = re.sub('\s+', '', soup.find(class_ ="wprm-recipe-servings-container").get_text()).partition(":")[2]
    except:
        servings = ''
    # if there is something like 3 to 4, take the average - do this later

    # total fat
    try:
        total_fat = [ings.get_text().partition('"fatContent":"')[2].partition(' g')[0]]
    except:
        total_fat = ''
        
    # saturated fat
    try:
        saturated_fat = [ings.get_text().partition('"saturatedFatContent":"')[2].partition(' g')[0]]
    except:
        saturated_fat = ''
        
    # fiber
    fiber = [ings.get_text().partition('"fiberContent":"')[2].partition(' g')[0]]

    # protein
    protein = [ings.get_text().partition('"proteinContent":"')[2].partition(' g')[0]]

    # sugar
    sugar = [ings.get_text().partition('"sugarContent":"')[2].partition(' g')[0]]

    # carbs
    carbs = [ings.get_text().partition('"carbohydrateContent":"')[2].partition(' g')[0]]

    # rating
    rating = [ings.get_text().partition('"ratingValue":"')[2].partition('",')[0]]
    
    # combine into df
    df = pd.DataFrame({
            
            'name': name,
            'description': description,
            'labels': labels,
            'photos': has_photos,
            'prep_time': prep_time,
            'cook_time': cook_time,
            'total_time': total_time,
            'calories': calories,
            'course': course,
            'cuisine': cuisine,
            'servings': servings,
            'total_fat': total_fat,
            'saturated_fat' : saturated_fat,
            'protein': protein,
            'sugar': sugar,
            'carbs': carbs,
            'rating': str(rating),
            'url': url
        })
    return name, df



In [331]:
master_url = "http://www.vegrecipesofindia.com/recipes-index/"
main_links = get_links(master_url)
master_links = []
for link in main_links:
    master_links += get_links(link)
links = list(set(master_links))

/Users/sushmitavgopalan/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/sushmitavgopalan/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


KeyboardInterrupt: 

In [482]:
def do_stuff(url):
    print(url)
    
    if "vegrecipesofindia" not in url:
        print("External Website")
        return
    
    if "comment" in url:
        print("No recipe found")
        return
    if ".com/hi" in url:
        print("Hindi. Ignoring")
        return
    
    try:
        page = requests.get(url)
    
    except:
        print("Sleeping for 30 seconds")
        time.sleep(30)

    if not page.status_code == 200:
        print("Page not found")
        return
    
    soup = BeautifulSoup(page.content, 'html.parser')
    recipe = soup.find(class_ ="wprm-recipe-ingredients-container")
    if recipe is None:
        print("No recipe found on this page")
        return
    print("Recipe Found")
    name, recipe_details = get_recipe_details(soup)
    ingredients = get_ingredients(recipe,name)

    return [name, recipe_details, ingredients]

In [490]:
recipes_df = pd.DataFrame({            
            'name': [''],
            'description': [''],
            'labels': [''],
            'photos': [''],
            'prep_time': [''],
            'cook_time': [''],
            'total_time': [''],
            'calories': [''],
            'course': [''],
            'cuisine': [''],
            'servings': [''],
            'total_fat': [''],
            'saturated_fat' : [''],
            'protein': [''],
            'sugar': [''],
            'carbs': [''],
            'rating': ['']
        })

ingredients_df =  pd.DataFrame({
            
            'name': [''],
            'amount': [''],
            'units': [''],
            'notes': [''],
            'recipe': ['']
    })

In [492]:
for k in range(len(links)):
    print(k)
    output = do_stuff(links[k])
    if output:
        recipes_df = recipes_df.append(output[1])
        ingredients_df = ingredients_df.append(output[2])
    

0
http://www.vegrecipesofindia.com/navratri-recipes-navratri-fasting-recipes/#comment-456076
No recipe found
1
http://www.vegrecipesofindia.com/chocolate-cake-recipe-in-pressure-cooker/#comment-565412
No recipe found
2
http://www.vegrecipesofindia.com/gobi-recipes-cauliflower-recipes/#comment-437276
No recipe found
3
http://www.vegrecipesofindia.com/gobi-paratha-punjabi-gobi-paratha/
Recipe Found
4
http://www.vegrecipesofindia.com/veg-sagu-recipe-veg-saagu/
Recipe Found
5
http://www.vegrecipesofindia.com/eggless-butter-cookies-recipe/#comment-551873
No recipe found
6
http://www.vegrecipesofindia.com/pasta-salad/
Recipe Found
7
http://www.vegrecipesofindia.com/eggless-cakes-recipes/#comment-630241
No recipe found
8
http://www.vegrecipesofindia.com/sweet-potato-chutney-recipe/
Recipe Found
9
http://www.vegrecipesofindia.com/kesari-bhath-recipe/
Recipe Found
10
http://www.vegrecipesofindia.com/recipes/north-indian-recipes/page/2/
No recipe found on this page
11
http://www.vegrecipesofindi

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No recipe found on this page
22
http://www.vegrecipesofindia.com/pav-bhaji-recipe-mumbai-pav-bhaji-a-fastfood-recipe-from-mumbai/
Recipe Found
23
http://www.vegrecipesofindia.com/rooh-afza-with-sabza-seeds/
Recipe Found
24
http://www.vegrecipesofindia.com/mix-veg-pulao-recipe-veg-pulao/
Recipe Found
25
http://www.vegrecipesofindia.com/chocolate-cake-recipe-in-pressure-cooker/#comment-629653
No recipe found
26
http://www.vegrecipesofindia.com/fruit-kesari-recipe/
Recipe Found
27
http://www.vegrecipesofindia.com/recipe/15-minutes-recipes/page/12/
No recipe found on this page
28
http://www.vegrecipesofindia.com/gobi-recipes-cauliflower-recipes/#comment-571787
No recipe found
29
http://www.vegrecipesofindia.com/chocolate-cake-recipe-in-pressure-cooker/#comment-558578
No recipe found
30
http://www.vegrecipesofindia.com/paneer-recipes-indian-paneer-recipes/#comment-416107
No recipe found
31
http://www.vegrecipesofindia.com/sweet-pongal-recipe-with-sugar/
Recipe Found
32
http://www.vegrecipes

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


34
http://www.vegrecipesofindia.com/wp-content/uploads/2017/03/ganga-jamuna-juice-recipe-11.jpg
No recipe found on this page
35
http://www.vegrecipesofindia.com/brinjal-sambar-kathirikai-sambar/
Recipe Found
36
http://www.vegrecipesofindia.com/sabudana-payasam-recipe/
Recipe Found
37
http://poornimastastytreats.blogspot.com/
External Website
38
http://www.vegrecipesofindia.com/aloo-rasedar-recipe/
Recipe Found
39
http://www.vegrecipesofindia.com/dill-leaves-indian-recipe-dill-leaves/
Recipe Found


...

No recipe found on this page
3398
http://www.vegrecipesofindia.com/no-cook-mango-kulfi-recipe/
Recipe Found
3399
http://www.vegrecipesofindia.com/eggless-butter-cookies-recipe/#comment-428205
No recipe found
3400
http://www.vegrecipesofindia.com/wp-content/uploads/2016/12/pressure-cooker-chocolate-cake-recipe34.jpg


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No recipe found on this page
3401
http://www.vegrecipesofindia.com/chocolate-cake-recipe-in-pressure-cooker/#comment-630237
No recipe found
3402
http://www.vegrecipesofindia.com/aamras-recipe/
Recipe Found
3403
http://www.vegrecipesofindia.com/ragi-halwa-recipe/
Recipe Found
3404
http://www.vegrecipesofindia.com/sandwich-recipes-veg-sandwich-recipes/#comment-570769
No recipe found
3405
http://www.vegrecipesofindia.com/chocolate-cake-recipe-in-pressure-cooker/#comment-622086
No recipe found
3406
http://www.vegrecipesofindia.com/bhindi-stir-fry-recipe
Recipe Found
3407
http://www.vegrecipesofindia.com/cookies-recipes-biscuit-recipes/#comments
No recipe found
3408
http://www.vegrecipesofindia.com/choliya-pulao-recipe-pulao-recipes/
Recipe Found
3409
http://vegrecipesofindia.com/mix-vegetable-recipe-with-paneer/
Recipe Found
3410
http://www.vegrecipesofindia.com/bombay-veg-grilled-sandwich-recipe/#comment-406618
No recipe found
3411
http://www.vegrecipesofindia.com/mushroom-recipes/#commen

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


No recipe found on this page
3414
http://www.vegrecipesofindia.com/mushroom-recipes/#comment-425242
No recipe found
3415
http://www.vegrecipesofindia.com/chocolate-recipes/
No recipe found on this page
3416
http://www.vegrecipesofindia.com/besan-toast-recipe-bread-besan-toast/
Recipe Found
3417
http://www.vegrecipesofindia.com/davangere-coconut-chutney-recipe/
Recipe Found
3418
http://www.vegrecipesofindia.com/green-peas-sundal-recipe/
Recipe Found
3419
http://www.vegrecipesofindia.com/whole-green-moong-dal-recipe/
Recipe Found
3420
http://www.vegrecipesofindia.com/mushroom-tikka/
Recipe Found


In [494]:
recipes_df

,calories,carbs,cook_time,course,cuisine,description,labels,name,photos,prep_time,protein,rating,saturated_fat,servings,sugar,total_fat,total_time,url
0,,,,,,,,,,,,,,,,,,NaN
0,,,25,breakfasts,"northindian,Punjabi",", how to make gobi paratha | punjabi gobi paratha","Kids Recipes, No Onion No Garlic, North India...",gobi paratha,True,25,,['4.5'],,12parathas,,,50,http://www.vegrecipesofindia.com/gobi-paratha-...
0,,,40,main,"karnataka,southindian",", how to make veg saagu recipe | mix veg sagu ...","Indian Curries, Popular Indian Recipes, Vegan ...",veg sagu,True,15,,['4.34'],,3to4,,,55,http://www.vegrecipesofindia.com/veg-sagu-reci...
0,,,15,side,Italian,", how to make easy pasta salad recipe | salad ...","Tiffin Recipes,30 Minutes Recipes, Starters &...",pasta salad,False,5,,[''],,3to4,,,20,http://www.vegrecipesofindia.com/pasta-salad/
0,,,,side,southindian,", south indian sweet potato chutney","No Onion No Garlic,30 Minutes Recipes, Vegan ...",sweet potato chutney,False,,,['4.5'],,4to6,,,,http://www.vegrecipesofindia.com/sweet-potato-...
0,,,15,sweets,"karnataka,southindian",", karnataka style rava kesari bhath recipe","Karnataka Recipes, Kids Recipes,30 Minutes Re...",kesari bhath,True,5,,['5'],,1to2,,,20,http://www.vegrecipesofindia.com/kesari-bhath-...
0,,,25,sweets,northindian,", how to make besan burfi recipe | barfi recipes","Kids Recipes,Festival Recipes,30 Minutes Reci...",besan barfi,False,5,,['3'],,1mediumbox,,,30,http://www.vegrecipesofindia.com/besan-barfi-r...
0,,,,Beverages,world,", how to make mango iced tea recipe","Summer Recipes,Beverages, Navratri & Fasting ...",mango iced tea,False,15,,['5'],,3glasses,,,15,http://www.vegrecipesofindia.com/mango-iced-tea/
0,,,25,"main,side",northindian,", how to make baby corn jalfrezi recipe","Main Course, North Indian Recipes, Vegan Reci...",baby corn jalfrezi,True,5,,['5'],,3to4,,,30,http://www.vegrecipesofindia.com/baby-corn-jal...
0,,,20,main,northindian,- how to make punjabi dahi wali arbi recipe,"Indian Curries, Main Course,Bachelor Recipes, ...",dahi wali arbi,False,20,,['5'],,4,,,40,http://www.vegrecipesofindia.com/dahi-wali-arbi/


In [468]:
a,b,c = do_stuff(a, recipes_df, ingredients_df)

http://www.vegrecipesofindia.com/gobi-paratha-punjabi-gobi-paratha/
Recipe Found


In [475]:
a


'gobi paratha '

In [478]:
b

,calories,carbs,cook_time,course,cuisine,description,labels,name,photos,prep_time,protein,rating,saturated_fat,servings,sugar,total_fat,total_time,url
0,,,25,breakfasts,"northindian,Punjabi",", how to make gobi paratha | punjabi gobi paratha","Kids Recipes, No Onion No Garlic, North India...",gobi paratha,True,25,,['4.5'],,12parathas,,,50,http://www.vegrecipesofindia.com/gobi-paratha-...
